<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/3step_lecture/Dense_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 10
batch_size = 32

### DenseUnit

In [0]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        self.concat = tf.keras.layers.Concatenate()

    def call(self, x, training=False, mask=None): # x: (Batch, H, W, Ch_in)
        h = self.bn(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv(h) # h: (Batch, H, W, filter_out)
        h = self.concat([x, h]) # (Batch, H, W, Ch_in + filter_out)
        return h

### DenseLayer

In [0]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence = list()
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=False)
        return x 

### Transition Layer

In [0]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.pool = tf.keras.layers.MaxPool2D()

    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        return self.pool(x)

### Model

In [0]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 28x28x8

        self.dl1 = DenseLayer(2, 4, (3, 3)) # 28x28x16
        self.tr1 = TransitionLayer(16, (3, 3)) # 14x14x16

        self.dl2 = DenseLayer(2, 8, (3, 3)) # 28x28x32
        self.tr2 = TransitionLayer(32, (3, 3)) # 7x7x32

        self.dl3 = DenseLayer(2, 16, (3, 3)) # 7x7x64

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        

    def call(self, x, training=False, mask=None):
        x = self.conv1(x)

        x = self.dl1(x, training=training)
        x = self.tr1(x)
        x = self.dl2(x, training=training)
        x = self.tr2(x)
        x = self.dl3(x, training=training)

        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x        

### Define train, test loop

In [0]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

### Load Data

In [8]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255., x_test / 255.
x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

11493376/11490434 [==============================] - 0s 0us/step


### Define model, loss, optimizer and metric

In [0]:
model = DenseNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [13]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result() * 100,
          test_loss.result(),
          test_accuracy.result() * 100
    ))

    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.112987220287323, Accuracy: 96.47166442871094, Test Loss: 0.048053301870822906, Test Accuracy: 98.47999572753906
Epoch 2, Loss: 0.04208860918879509, Accuracy: 98.70999908447266, Test Loss: 0.04376764968037605, Test Accuracy: 98.56999969482422
Epoch 3, Loss: 0.03177628293633461, Accuracy: 98.98999786376953, Test Loss: 0.03924807161092758, Test Accuracy: 98.79999542236328
Epoch 4, Loss: 0.023990724235773087, Accuracy: 99.25167083740234, Test Loss: 0.033345095813274384, Test Accuracy: 99.01000213623047
Epoch 5, Loss: 0.020268982276320457, Accuracy: 99.34832763671875, Test Loss: 0.035146862268447876, Test Accuracy: 98.98999786376953
Epoch 6, Loss: 0.017092781141400337, Accuracy: 99.42500305175781, Test Loss: 0.040640562772750854, Test Accuracy: 98.91999816894531
Epoch 7, Loss: 0.014760964550077915, Accuracy: 99.52666473388672, Test Loss: 0.03548755124211311, Test Accuracy: 99.11000061035156
Epoch 8, Loss: 0.014561054296791553, Accuracy: 99.55166625976562, Test Loss: 0.04321

In [14]:
model.summary()

Model: "dense_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            multiple                  80        
_________________________________________________________________
dense_layer_3 (DenseLayer)   multiple                  808       
_________________________________________________________________
transition_layer_2 (Transiti multiple                  2320      
_________________________________________________________________
dense_layer_4 (DenseLayer)   multiple                  3056      
_________________________________________________________________
transition_layer_3 (Transiti multiple                  9248      
_________________________________________________________________
dense_layer_5 (DenseLayer)   multiple                  11872     
_________________________________________________________________
flatten_1 (Flatten)          multiple                  